# AltaML Technical Task

In this notebook, we will be exploring the problem given by AltaML and answer some of the questions provided. Let us first begin by creating a cell to import all the packages that will be used in this project:

In [1]:
import numpy as np
import pandas as pd

Let's take a first look at the data: